In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
import csv
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import re
import pandas as pd
import re
import nltk
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
import unicodedata as ud
# import optuna

import csv
import networkx as nx
import numpy as np
from sklearn.linear_model import LogisticRegression


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, fowlkes_mallows_score
from sklearn.decomposition import PCA
from io import BytesIO
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer,TFAutoModel
from nltk.tokenize import word_tokenize
from random import choice
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
# file_path = os.path.join(os.path.expanduser("~"), "Desktop", "data_challenge", "train.txt")
# read training data
train_domains = list()
y_train = list()
with open("/content/drive/Othercomputers/My Computer/Masters_Staff/trimester_3/Data_challenge/project/data/train.txt", 'r') as f:
# with open("../../../../../data_challenge/train.txt", 'r') as f:
    for line in f:
        l = line.split(',') # domain names
        train_domains.append(l[0])
        y_train.append(l[1][:-1]) # topics of domain names

# read test data
test_domains = list()
with open("/content/drive/Othercomputers/My Computer/Masters_Staff/trimester_3/Data_challenge/project/data/test.txt", 'r') as f:
# with open("../../../../../data_challenge/test.txt", 'r') as f:
    for line in f:
        l = line.split(',')
        test_domains.append(l[0])

# create a directed graph
G = nx.read_edgelist('/content/drive/Othercomputers/My Computer/Masters_Staff/trimester_3/Data_challenge/project/data/edgelist.txt', delimiter=' ',
#                       create_using=nx.DiGraph())

# G = nx.read_edgelist('../../../../../data_challenge/edgelist.txt', delimiter=' ',
                      create_using=nx.DiGraph())

print('Number of nodes:', G.number_of_nodes())
print('Number of edges:', G.number_of_edges())


# read textual content of webpages of domain names
text = dict()
with zipfile.ZipFile('/content/drive/Othercomputers/My Computer/Masters_Staff/trimester_3/Data_challenge/project/data/domains.zip', "r") as zfile:

# with zipfile.ZipFile('../../../../../data_challenge/domains.zip', "r") as zfile:
    for filename in zfile.namelist():
        if re.search(r'\.zip$', filename) is not None:
            zfiledata = BytesIO(zfile.read(filename))
            with zipfile.ZipFile(zfiledata) as zfile2:
                text[filename[:-4]] = ''
                for name2 in zfile2.namelist():
                    file = zfile2.read(name2)
                    text[filename[:-4]] += file.decode('utf16') + ' '

# retrieve textual content of domain names of the training set
train_data = list()
for domain in train_domains:
    if domain in text:
        train_data.append(text[domain])
    else:
        train_data.append('')


# retrieve textual content of domain names of the test set
test_data = list()
for domain in test_domains:
    if domain in text:
        test_data.append(text[domain])
    else:
        test_data.append('')

# to reduce memory
text = None

Number of nodes: 65208
Number of edges: 1642073


### Create validation set

In [68]:
train_df = pd.DataFrame({'domain':train_domains, 'y':y_train})


In [69]:
from sklearn.model_selection import train_test_split

X_train, X_dev, y_train, y_dev = train_test_split(train_df['domain'], train_df['y'], test_size=0.20, random_state=4321)

print(X_train.shape, X_dev.shape, y_train.shape, y_dev.shape)

(1449,) (363,) (1449,) (363,)


### Create long tensor for GraphSage

In [122]:
# Initialize a boolean tensor with False for all nodes
train_nodes_tensor = torch.zeros(G.number_of_nodes(), dtype=torch.bool)

# Iterate over the nodes in the graph
for idx, node in enumerate(G.nodes()):
    # Check if the node is in train_domains
    if node in X_train:
        train_nodes_tensor[idx] = True

print(train_nodes_tensor)

tensor([False, False, False,  ..., False, False, False])


### Community

In [ ]:
# coding=utf-8


class Status(object):
    """
    To handle several data in one struct.

    Could be replaced by named tuple, but don't want to depend on python 2.6
    """
    node2com = {}
    total_weight = 0
    internals = {}
    degrees = {}
    gdegrees = {}

    def __init__(self):
        self.node2com = dict([])
        self.total_weight = 0
        self.degrees = dict([])
        self.gdegrees = dict([])
        self.internals = dict([])
        self.loops = dict([])

    def __str__(self):
        return ("node2com : " + str(self.node2com) + " degrees : "
                + str(self.degrees) + " internals : " + str(self.internals)
                + " total_weight : " + str(self.total_weight))

    def copy(self):
        """Perform a deep copy of status"""
        new_status = Status()
        new_status.node2com = self.node2com.copy()
        new_status.internals = self.internals.copy()
        new_status.degrees = self.degrees.copy()
        new_status.gdegrees = self.gdegrees.copy()
        new_status.total_weight = self.total_weight

    def init(self, graph, weight, part=None):
        """Initialize the status of a graph with every node in one community"""
        count = 0
        self.node2com = dict([])
        self.total_weight = 0
        self.degrees = dict([])
        self.gdegrees = dict([])
        self.internals = dict([])
        self.total_weight = graph.size(weight=weight)
        if part is None:
            for node in graph.nodes():
                self.node2com[node] = count
                deg = float(graph.degree(node, weight=weight))
                if deg < 0:
                    error = "Bad node degree ({})".format(deg)
                    raise ValueError(error)
                self.degrees[count] = deg
                self.gdegrees[node] = deg
                edge_data = graph.get_edge_data(node, node, default={weight: 0})
                self.loops[node] = float(edge_data.get(weight, 1))
                self.internals[count] = self.loops[node]
                count += 1
        else:
            for node in graph.nodes():
                com = part[node]
                self.node2com[node] = com
                deg = float(graph.degree(node, weight=weight))
                self.degrees[com] = self.degrees.get(com, 0) + deg
                self.gdegrees[node] = deg
                inc = 0.
                for neighbor, datas in graph[node].items():
                    edge_weight = datas.get(weight, 1)
                    if edge_weight <= 0:
                        error = "Bad graph type ({})".format(type(graph))
                        raise ValueError(error)
                    if part[neighbor] == com:
                        if neighbor == node:
                            inc += float(edge_weight)
                        else:
                            inc += float(edge_weight) / 2.
                self.internals[com] = self.internals.get(com, 0) + inc

# class usage
status = Status()
status.init(G,1642073)
# status.node2com

In [ ]:
status.internals

In [ ]:
### Find-out community number
def cluster_keys_by_value(input_dict):
    clusters = {}

    # reverse the input dictionary to create a new dictionary where values become keys
    reversed_dict = {}
    for key, value in input_dict.items():
        reversed_dict.setdefault(value, []).append(key)

    # iterate over unique values in the reversed dictionary
    for value, keys in reversed_dict.items():
        # add keys associated with the same value to the cluster
        clusters[value] = keys

    # filter and sort to find communities
    filtered_clusters = {value: keys for value, keys in clusters.items() if len(keys)>1}
    sorted_clusters = sorted_comms = sorted(filtered_clusters.items(),
                                            key=lambda kv: len(kv[1]), reverse=True)
    return sorted_clusters

In [ ]:
status = Status() # class object

# check a range of values for weight
for weight in range(-1000, 1000):

  status.init(G, weight)
  sorted_comms = cluster_keys_by_value(status.node2com)

  if sorted_comms:
    print(f"Communities can be build with weight: {weight}")
    break

In [ ]:
len(y_train),len(train_domains)

(1812, 1812)

In [ ]:
## here subgraph will be the developement set

# create a subgraph
num_nodes_sub = int(0.01 * len(G))
sub_nodes = list(G.nodes())[:num_nodes_sub] # subset of nodes
subg = G.subgraph(sub_nodes)

# retrieve domains && labels
subg_domains = [domain for domain in train_domains if domain in subg.nodes()]
ydev_sub = [y_train[train_domains.index(node)] for node in subg.nodes() if node in train_domains]

# re-create the big part of the net
new_G = G.copy()
new_G.remove_nodes_from(subg.nodes())
new_train_dom = [domain for domain in train_domains if domain not in subg_domains]
ytr = [y_train[train_domains.index(node)] for node in new_G.nodes() if node in train_domains]

In [ ]:
len(ydev_sub),subg.number_of_nodes(), len(subg_domains)

(147, 652, 147)

### DEEPWALK

In [93]:
def random_walk(G, node, walk_length):
    # Starts from vertex "node" and performs a random walk of length "walk length". Returns a list of the visited vertices
    walk = [node]

    for _ in range(walk_length-1):
        neighbors = list(G.neighbors(walk[-1]))
        if len(neighbors) > 0:
            neighbor = choice(neighbors)
            walk.append(neighbor)
        else:
            break

    walk = [str(node) for node in walk]
    return walk


def generate_walks(G, num_walks, walk_length):
    # Runs "num_walks" random walks from each node, and returns a list of all random walk
    walks = []

    for _ in range(num_walks):
        for node in G.nodes():
            walk = random_walk(G, node, walk_length)
            walks.append(walk)

    return walks


graph_walks = generate_walks(G, 10, 40)

# initialize word2vec object
model = Word2Vec(graph_walks, vector_size=128, window=10, min_count=0, sg=1, workers=8, epochs=5)


# function to generate node embeddings based on deepwalk and w2v
def deepwalk_to_vector(data_domains,w2v_model,dims=128):

  #  Each row corresponds to a web host.
  embs = np.zeros((len(data_domains), dims)) # create the training matrix.
  for i, domain in enumerate(data_domains):
      embs[i,:] = model.wv[domain]

  return embs

# generate embeddings
Xtr_emb = deepwalk_to_vector(X_train,model)
Xde_emb = deepwalk_to_vector(X_dev,model)
Xte_emb = deepwalk_to_vector(test_domains,model)


In [94]:
def multiclass_cross_entropy(y_true, y_pred_prob):
    epsilon = 1e-15  # small value to prevent log(0)
    # clip predicted probabilities to avoid log(0)
    y_pred_prob = np.clip(y_pred_prob, epsilon, 1 - epsilon)
    # compute cross-entropy loss

    loss = -np.mean(np.sum(y_true * np.log(y_pred_prob), axis=1))
    return loss

In [95]:
import tensorflow as tf
ytrain_en = tf.keras.utils.to_categorical(y_train, num_classes=9)
ydev_en= tf.keras.utils.to_categorical(y_dev,num_classes=9)

In [96]:
# use logistic regression to classify the webpages of the test set
clf = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=1500)
clf.fit(Xtr_emb, y_train)
y_pred_tr = clf.predict_proba(Xtr_emb)
y_pred_de = clf.predict_proba(Xde_emb)


ce_train = multiclass_cross_entropy(ytrain_en, y_pred_tr)
ce_dev = multiclass_cross_entropy(ydev_en, y_pred_de)

print(f"Cross Entropy Loss for Train: {ce_train}")
print(f"Cross Entropy Loss for Dev: {ce_dev}")

Cross Entropy Loss for Train: 0.8483325005134608
Cross Entropy Loss for Dev: 1.1640980708234865


In [74]:
%%capture
!pip install optuna

In [ ]:
import optuna
def objective(trial, X, y):
    # define hyperparameters to optimize
    C = trial.suggest_float('C', 0.01, 100, log=True)
    penalty = trial.suggest_categorical('penalty', ['l2'])
    max_iter = trial.suggest_int('max_iter', 1000, 5000)

    model = LogisticRegression(C=C, penalty=penalty, max_iter=max_iter, solver='liblinear')

    # perform cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')

    # return average accuracy as the objective value
    return scores.mean()

# begin studies
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, Xtr_emb, y_train), n_trials=40)

best_params = study.best_params
best_score = study.best_value

print("Best parameters:", best_params)

print("Best score:", best_score)


In [98]:
best_lr = LogisticRegression(**best_params)
best_lr.fit(Xtr_emb, y_train)

yde_b_lr = best_lr.predict_proba(Xde_emb)
ce_dev = multiclass_cross_entropy(ydev_en, yde_b_lr)
print(f"Cross Entropy Loss for Dev: {ce_dev}")

Cross Entropy Loss for Dev: 1.1258391085185928


### GraphSage

In [126]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='.', name="Pubmed")
data = dataset[0]

print(type(dataset[0]))
# # Print information about the dataset
# print(f'Dataset: {dataset}')
# print('-------------------')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of nodes: {data.x.shape[0]}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

# # Print information about the graph
# print(f'\nGraph:')
# print('------')
# print(f'Training nodes: {sum(data.train_mask).item()}')
# print(f'Evaluation nodes: {sum(data.val_mask).item()}')
# print(f'Test nodes: {sum(data.test_mask).item()}')
# print(f'Edges are directed: {data.is_directed()}')
# print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
# print(f'Graph has loops: {data.has_self_loops()}')

<class 'torch_geometric.data.data.Data'>


In [166]:
y_train = torch.tensor( np.array(y_train).astype('int64') )
y_dev = torch.tensor( np.array(y_dev).astype('int64') )

In [168]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, input_nodes, labels):
        self.input_nodes = input_nodes
        self.labels = labels

    def __len__(self):
        return len(self.input_nodes)

    def __getitem__(self, idx):
        return self.input_nodes[idx], self.labels[idx]

tr_dataset = CustomDataset(train_nodes_tensor, y_train)
data = pyg_utils.from_networkx(G) # transform nx graph torch_geometric.data.data.Data

In [170]:
train_nodes_tensor.shape

torch.Size([65208])

In [158]:
type(data)

torch_geometric.data.data.Data

In [169]:
from torch_geometric.loader import NeighborLoader
from torch_geometric.utils import to_networkx
import torch_geometric.utils as pyg_utils



# Create batches with neighbor sampling
train_loader = NeighborLoader(
    data,
    num_neighbors=[5, 10],
    batch_size=16,
    input_nodes=tr_dataset,
)

# Print each subgraph
for i, subgraph in enumerate(train_loader):
    print(f'Subgraph {i}: {subgraph}')

# Plot each subgraph
fig = plt.figure(figsize=(16,16))
for idx, (subdata, pos) in enumerate(zip(train_loader, [221, 222, 223, 224])):
    G = to_networkx(subdata, to_undirected=True)
    ax = fig.add_subplot(pos)
    ax.set_title(f'Subgraph {idx}')
    plt.axis('off')
    nx.draw_networkx(G,
                    pos=nx.spring_layout(G, seed=0),
                    with_labels=True,
                    node_size=200,
                    node_color=subdata.y,
                    cmap="cool",
                    font_size=10
                    )
plt.show()

/usr/local/lib/python3.10/dist-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


ValueError: expected sequence of length 65208 at dim 0 (got 1449)

In [153]:
for batch in train_loader:
  print(batch.y)


In [144]:
import torch
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from torch_geometric.datasets import Planetoid

class GraphSAGE(torch.nn.Module):
  """GraphSAGE"""
  def __init__(self, dim_in, dim_h, dim_out):
    super().__init__()
    self.sage1 = SAGEConv(dim_in, dim_h)
    self.sage2 = SAGEConv(dim_h, dim_out)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.01,
                                      weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = self.sage1(x, edge_index)
    h = torch.relu(h)
    h = F.dropout(h, p=0.5, training=self.training)
    h = self.sage2(h, edge_index)
    return h, F.log_softmax(h, dim=1)

  def fit(self, train_loader, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = self.optimizer

    self.train()
    for epoch in range(epochs+1):
        acc = 0
        val_loss = 0
        val_acc = 0

        # Train on batches
        for batch in train_loader:
            optimizer.zero_grad()
            _, out = self(batch.x, batch.edge_index)
            loss = criterion(out[batch.train_mask], batch.y[batch.train_mask])
            acc += accuracy(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
            loss.backward()
            optimizer.step()

            # Validation
            val_loss += criterion(out[batch.val_mask], batch.y[batch.val_mask])
            val_acc += accuracy(out[batch.val_mask].argmax(dim=1), batch.y[batch.val_mask])

        # # Print metrics every 10 epochs
        # if(epoch % 10 == 0):
        #     print(f'Epoch {epoch:>3} | Train Loss: {loss/len(train_loader):.3f} '
        #           f'| Train Acc: {acc/len(train_loader)*100:>6.2f}% | Val Loss: '
        #           f'{val_loss/len(train_loader):.2f} | Val Acc: '
        #           f'{val_acc/len(train_loader)*100:.2f}%')



# GraphSage instance
gSage = GraphSAGE(Xtr_emb.shape[1], 100, 9)

# train graphsage
gSage.fit(train_loader,100)

In [ ]:
#### Get Edge Index Array
def edge_list(graph):
  edge_list = list(graph.edges)
  edge_index_matrix = np.array(edge_list).T
  return edge_index_matrix

edge_list  = edge_list(G)

# # re-create the big part of the net
# tr_G = G.copy()
# tr_G.remove_nodes_from(subg.nodes())
# new_train_dom = [domain for domain in train_domains if domain not in subg_domains]
# ytr = [y_train[train_domains.index(node)] for node in new_G.nodes() if node in train_domains]

### GCN on Graph (Pytorch Geometric)

In [ ]:
%%capture
%pip install torch_geometric
import torch_geometric
import torch_geometric.nn as g_nn
import torch_geometric.data as g_data

from torch_geometric.nn import GCNConv

In [ ]:
num_class = 9

class GCN(torch.nn.Module):
  def __init__(self, hidden_channels):
    super().__init__()
    torch.manual_seed(1234567)
    self.conv1 = GCNConv(w2v_train.shape[1], hidden_channels)
    self.conv2 = GCNConv(hidden_channels, hidden_channels)
    self.conv3 = GCNConv(hidden_channels, hidden_channels)
    self.conv4 = GCNConv(hidden_channels, num_class)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = x.relu()
    x = F.dropout(x, p=0.3, training=self.training)
    x = self.conv2(x, edge_index)
    x = x.relu()
    x = F.dropout(x, p=0.3, training=self.training)
    x = self.conv3(x, edge_index)
    x = F.dropout(x, p=0.3, training=self.training)
    x = self.conv4(x, edge_index)
    return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(128, 16)
  (conv2): GCNConv(16, 16)
  (conv3): GCNConv(16, 16)
  (conv4): GCNConv(16, 9)
)


In [ ]:
w2v_train = .astype('float64')

In [ ]:
w2v_train = torch.tensor(w2v_train)

In [ ]:
w2v_train.shape

torch.Size([1812, 128])

In [ ]:
from sklearn.preprocessing import LabelEncoder

# reshape array to single list of strs
edge_list_flat = edge_list.flatten()

# initialize label encoder and transform
label_encoder = LabelEncoder()
num_instances = label_encoder.fit_transform(edge_list_flat)
num_instances = num_instances.reshape(edge_list.shape)# reshape to og shape

In [ ]:
num_instances = torch.tensor( num_instances.astype('int64') )

In [ ]:
model = GCN(hidden_channels=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

losses,eps_list = [], []

def train():
  model.train()
  optimizer.zero_grad() # clear gradients
  out = model(X, num_instances) # perform a single forward pass
  # print(f"Output og shape: {out.shape}")
  out = F.softmax(out,dim=1) # pass through softmax func
  # print(f"Output after-softmax shape: {out.shape}")
  loss = criterion(out, ytrain_en.float()) # compute the loss solely based on train nodes
  losses.append(loss.item())
  eps_list.append(epoch)
  loss.backward() # derive gradients
  optimizer.step() # update params
  return loss

def dev():
  model.eval()
  out = model(flat_emb_de, edge_de)
  out = F.softmax(out,dim=1) # use the class with highest prob
  dev_loss = criterion(out, ydev_en.float()) # derive ratio of correct predictions
  return dev_loss

for epoch in range(1, 10001):
  loss = train()
  # print(f"Epoch: {epoch:03d}, loss: {loss:.4f}")


RuntimeError: index 22411 is out of bounds for dimension 0 with size 1812

In [ ]:
dev_loss = dev()
print(f'Dev CE-Loss: {dev_loss:.4f}')

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(eps_list, losses)

In [ ]:
%%capture
!pip install torchviz
!sudo apt-get install graphviz

In [ ]:
from torchviz import make_dot
from torchsummary import summary

y =  model(flat_emb_tr, edge_tr)
make_dot(y, params=dict(model.named_parameters()))

### ENSEMBLES

In [ ]:
# -- combine probabilities of all the models we want
probs_train = np.stack([ytr_b_xgb,ytr_xgb], axis=2)

avg_probs_tr = np.mean(probs_train, axis=2) # average probability for each class all clfs
avg_probs_tr.shape

(1812, 9)

In [ ]:
ensemble_cross_en = multiclass_cross_entropy(ytrain_en, avg_probs_tr)
ensemble_cross_en

0.15567404

#### ENSEMBLES predict on TEST

In [ ]:
ytest_xgb = Xbg.predict_proba(w2v_test)
ytest_b_xgb = Xbg.predict_proba(w2v_test)


probs_test = np.stack([ytest_b_xgbytest_xgb, e], axis=2)

avg_probs_test = np.mean(probs_test, axis=2) # average probability for each class all clfs
avg_probs_test.shape

(605, 9)

In [ ]:
# Write predictions to a file
with open('deepwalk_xgb.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    lst = list()
    for i in range(9):
        lst.append('class_'+str(i))
    lst.insert(0, "domain_name")
    writer.writerow(lst)
    for i,test_domain in enumerate(test_domains):
        lst = ytest_b_xgb[i,:].tolist()
        lst.insert(0, test_domain)
        writer.writerow(lst)